In [ ]:
!pip install geopandas
!pip install geopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import math
import geopandas as gpd
import pandas as pd
from shapely.geometry import MultiPolygon
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim

import folium
from folium import Choropleth, Marker
from folium.plugins import HeatMap, MarkerCluster

# Immigrant Resident Data

In [ ]:
#connect to Gdrive
try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

if IN_COLAB:
  print("Attempting to attach your Google gDrive to this Colab Jupyter Notebook")
  drive.mount('/gdrive')
else:
  print("Your Google gDrive is attached to this Colab Jupyter Notebook")

Attempting to attach your Google gDrive to this Colab Jupyter Notebook
Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
# Change directory
%cd /gdrive/MyDrive/"SF Geospatial Analysis"/
!pwd

[Errno 2] No such file or directory: '/gdrive/MyDrive/SF Geospatial Analysis/'
/content
/content


In [ ]:
# Read in SF neighborhoods file
districts = gpd.read_file("sf_neighborhoods.csv")
districts.head()

,WKT,SHAPE_Length,NEIGHBORHOOD,GLOBALID,Area,Length,geometry
0,"POLYGON Z ((-122.424756486 37.7478493880001 0,...",0.0871865741754845,Mission,{50D5BCF3-EE7D-4222-9F9A-2369E4BB31DC},,,"POLYGON Z ((-122.42476 37.74785 0.00000, -122...."
1,"POLYGON Z ((-122.408912333 37.7901316510001 0,...",0.0597217696795149,Downtown/Civic Center,{25DB5DF4-DE12-457A-86FF-68A61CEE59FA},,,"POLYGON Z ((-122.40891 37.79013 0.00000, -122...."
2,MULTIPOLYGON Z (((-122.388108237 37.74780316 0...,0.377675609396014,Bayview,{360F5CF4-7127-49D2-92A9-BE9A04B0B1A3},,,"MULTIPOLYGON Z (((-122.38811 37.74780 0.00000,..."
3,"POLYGON Z ((-122.42085002 37.7083345650001 0,-...",0.07158856555919,Crocker Amazon,{C2DC247E-DE56-48D4-BDFD-882B0BBD5C1B},,,"POLYGON Z ((-122.42085 37.70833 0.00000, -122...."
4,"POLYGON Z ((-122.42418272 37.742261677 0,-122....",0.0431664734207246,Glen Park,{418AF9A8-481A-4CEA-A341-9FA4A8F381DD},,,"POLYGON Z ((-122.42418 37.74226 0.00000, -122...."


In [ ]:
# isolate Mission District polygon and place in dictionary
districts_dict = districts[["NEIGHBORHOOD", "geometry"]].set_index("NEIGHBORHOOD")
districts_dict.head()
mission_df = districts[districts["NEIGHBORHOOD"] == "Mission"]
mission_dict = mission_df[["NEIGHBORHOOD", "geometry"]].set_index("NEIGHBORHOOD")
mission_dict.head()

,geometry
NEIGHBORHOOD,
Mission,"POLYGON Z ((-122.42476 37.74785 0.00000, -122...."


In [ ]:
# import NGOs in California dataset
cali_NGO = gpd.read_file('CaliLatAmNonProfits.csv')
cali_NGO.head()

,Latino Nonprofit Name,Street Address 1,Street Address 2,City,State,Zip/Postal Code,Latitude,Longitude,Phone Number,Website,Main Focus,Mission,geometry
0,"10,000 Degrees",120 Stony Point Rd,STE 150,Santa Rosa,CA,95401,38.442256,-122.7445519,415-459-4240,http://www.10000degrees.org,Education,To help students from low-income backgrounds g...,None
1,240 Shotwell Street,4434 Mission Street,,San Francisco,CA,94112,37.7268243,-122.4335194,415-406-1353,www.mnhc.org,Health,To honor Latino roots with a tradition of prov...,None
2,67 Sueños,65 9th St,,San Francisco,CA,94103,37.7767715,-122.4150038,415-565-0201,https://www.afsc.org/program/67-sue%C3%B1os,Leadership Development & Youth Empowerment,To advocate for migrant communities impacted b...,None
3,99 Rootz (Program of Power California),1637 M St.,,Merced,CA,95340,37.3018307,-120.4851852,209-626-0631,https://www.facebook.com/99Rootz/,Civic Engagement,To build a youth movement that is lead by youn...,None
4,Abriendo Puertas/Opening Doors (Project of Tid...,1000 N. Alameda St,STE 240,Los Angeles,CA,90012,34.0594561,-118.2359125,213-346-3216,http://www.ap-od.org/,Education,To engage parents in lessons that draw from re...,None


In [ ]:
# filter Cali NGOs into SF NGOS
sf_NGO = cali_NGO[cali_NGO['City'] == 'San Francisco'] 
sf_NGO.head()

,Latino Nonprofit Name,Street Address 1,Street Address 2,City,State,Zip/Postal Code,Latitude,Longitude,Phone Number,Website,Main Focus,Mission,geometry
1,240 Shotwell Street,4434 Mission Street,,San Francisco,CA,94112,37.7268243,-122.4335194,415-406-1353,www.mnhc.org,Health,To honor Latino roots with a tradition of prov...,None
2,67 Sueños,65 9th St,,San Francisco,CA,94103,37.7767715,-122.4150038,415-565-0201,https://www.afsc.org/program/67-sue%C3%B1os,Leadership Development & Youth Empowerment,To advocate for migrant communities impacted b...,None
7,Accion Latina,2958 24th St,,San Francisco,CA,94110,37.7527948,-122.4113092,415-648-1045,http://www.accionlatina.org/,Arts & Culture,"To promote cultural arts, community media, and...",None
10,Aguilas,1800 Market St,3rd floor Rm. Q32,San Francisco,CA,94102,37.7716881,-122.4240297,415-558-8403,http://sfaguilas.org,LGBTQ Rights,"To create a supportive, culturally sensitive e...",None
26,Bay Area Central American Chamber of Commerce,177 Lobos Street,,San Francisco,CA,94112,37.7147125,-122.4583897,650-773-6197,http://www.cahcc.com/Membership/LocalChambersA...,Economic Development,To represent the business interests of over 80...,None


In [ ]:
# Create map Zoomed in on Mission District, SF
m_2 = folium.Map(location=[37.76, -122.42], zoom_start=15)

# Visualize Mission District
for _, r in mission_df.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    geo_j.add_to(m_2)

# Visualize the SF NGO locations
for idx, row in sf_NGO.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row["Latino Nonprofit Name"],icon=folium.Icon(color='green')).add_to(m_2)

# Show the map
m_2

# Western Union Locations using Geolocator


In [ ]:
geolocator = Nominatim(user_agent="Reserve x Western Union", timeout=2)

In [ ]:
WULoc_df = pd.read_csv("SFWesternUnionLoc.csv")
WULoc_df .head()

,Name,Address
0,Western Union,"100 Upper Terr, San Francisco, CA, 94117"
1,Western Union,"199 Parnassus Ave, San Francisco, CA, 94117"
2,Western Union,"498 Castro St, San Francisco, CA, 94114"
3,Western Union,"443 Castro St, San Francisco, CA, 94114"
4,Western Union,"3998 18th St, San Francisco, CA, 94114"


In [ ]:
# debugging
print(WULoc_df['Address'].loc[WU_df.index[0]])
  
location = geolocator.geocode(WULoc_df['Address'].loc[WULoc_df.index[5]])
location

199 Parnassus Ave, San Francisco, CA, 94117


Location(1333, Castro Street, Noe Valley, San Francisco, CAL Fire Northern Region, California, 94114, United States, (37.75085295, -122.43368645, 0.0))

In [ ]:
WULat = []
WULon = []
for index, row in WULoc_df.iterrows():
  location = geolocator.geocode(WULoc_df['Address'].loc[WULoc_df.index[index]])
  WULat.append(location.latitude)
  WULon.append(location.longitude)

WULoc_df["Latitude"] = WULat
WULoc_df["Longitude"] = WULon

WULoc_df.head()

,Name,Address,Latitude,Longitude
0,Western Union,"100 Upper Terr, San Francisco, CA, 94117",37.765371,-122.443814
1,Western Union,"199 Parnassus Ave, San Francisco, CA, 94117",37.764405,-122.452336
2,Western Union,"498 Castro St, San Francisco, CA, 94114",37.761010,-122.435204
3,Western Union,"443 Castro St, San Francisco, CA, 94114",37.761829,-122.434842
4,Western Union,"3998 18th St, San Francisco, CA, 94114",37.761179,-122.432574


In [ ]:
# Gather Western Union Locations in San Francisco using geolocator, storing results in WU_df
possible_locations = geolocator.geocode("Western Union, San Francisco", exactly_one=False)

WU_df = pd.DataFrame(columns=['Name', 'Latitude', 'Longitude'])
index=1
for location in possible_locations:
     WU_df.at[index, 'Name'] = location.address
     WU_df.at[index, 'Latitude'] = location.latitude
     WU_df.at[index, 'Longitude'] = location.longitude
     index += 1

WU_df.head()

,Name,Latitude,Longitude
1,"Western Union, Mission Street, Mission Distric...",37.764872,-122.419517
2,"Western Union, Eddy Street, Tenderloin, San Fr...",37.783377,-122.417931
3,"Western Union, Post Street, Tenderloin, San Fr...",37.787019,-122.419804
4,"Western Union, Palou Avenue, Bayview District,...",37.733722,-122.390814
5,"Western Union, Roxas, Poblacion, San Francisco...",8.506614,125.978615


In [ ]:
# Create map Zoomed in on Mission District, SF
m_9 = folium.Map(location=[37.76, -122.42], zoom_start=15)

# Visualize Mission District
for _, r in mission_df.iterrows():
    # Without simplifying the representation of district,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    geo_j.add_to(m_9)

# Visualize the Western Union locations
for idx, row in WULoc_df.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row["Name"],icon=folium.Icon(color='blue')).add_to(m_9)

# Visualize Possible Western Union location
for idx, row in WU_df.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup="Possibly Western Union?",icon=folium.Icon(color='gray')).add_to(m_9)

# Show the map
m_9

Western Union Money Transfer Locations

In [ ]:
WUMoneyTransferLoc_df = pd.read_csv("SFWUMoneyTransferLocsDF.csv")
WUMoneyTransferLoc_df.head()

,Name,Street Address
0,CA CHECK CASHING #574,"1098 Market St, San Francisco, CA, 94102-3923"
1,WALGREENS #4609,"1301 Market St, San Francisco, CA, 94103-1307"
2,MONEY MART #1951,"500 Valencia St, San Francisco, CA, 94110-1115"
3,SAFEWAY #1490,"2300 16th St, San Francisco, CA, 94103-4805"
4,SAFEWAY #1507,"2020 Market St, San Francisco, CA, 94114-1314"


In [ ]:
print(WUMoneyTransferLoc_df['Street Address'].loc[WUMoneyTransferLoc_df.index[0]])

1098 Market St, San Francisco, CA, 94102-3923


In [ ]:
WUMTLat = []
WUMTLon = []
for index, row in WUMoneyTransferLoc_df.iterrows():
  location = geolocator.geocode(WUMoneyTransferLoc_df['Street Address'].loc[WUMoneyTransferLoc_df.index[index]])
  WUMTLat.append(location.latitude)
  WUMTLon.append(location.longitude)

WUMoneyTransferLoc_df["Latitude"] = WUMTLat
WUMoneyTransferLoc_df["Longitude"] = WUMTLon

WUMoneyTransferLoc_df.head()

,Name,Street Address,Latitude,Longitude
0,CA CHECK CASHING #574,"1098 Market St, San Francisco, CA, 94102-3923",37.781459,-122.411677
1,WALGREENS #4609,"1301 Market St, San Francisco, CA, 94103-1307",37.777256,-122.416227
2,MONEY MART #1951,"500 Valencia St, San Francisco, CA, 94110-1115",37.764692,-122.422161
3,SAFEWAY #1490,"2300 16th St, San Francisco, CA, 94103-4805",37.765749,-122.409471
4,SAFEWAY #1507,"2020 Market St, San Francisco, CA, 94114-1314",37.768891,-122.428442


In [ ]:
# Create map Zoomed in on Mission District, SF
m_4 = folium.Map(location=[37.76, -122.42], zoom_start=15)

# Visualize Mission District
for _, r in mission_df.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    geo_j.add_to(m_4)

# Visualize the Western Union Money Transfer locations
for idx, row in WUMoneyTransferLoc_df.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=WUMoneyTransferLoc_df['Name'].loc[WUMoneyTransferLoc_df.index[idx]], icon=folium.Icon(color='red')).add_to(m_4)

# Show the map
m_4

In [ ]:
# Create map Zoomed in on Mission District, SF
m_10 = folium.Map(location=[37.76, -122.42], zoom_start=15)

# Visualize Mission District
for _, r in mission_df.iterrows():
    # Without simplifying the representation of district,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    geo_j.add_to(m_10)

# Visualize the Western Union locations
for idx, row in WULoc_df.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row["Name"],icon=folium.Icon(color='blue')).add_to(m_10)

# Visualize Possible Western Union location
for idx, row in WU_df.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup="Possibly Western Union?",icon=folium.Icon(color='gray')).add_to(m_10)

# Visualize the Western Union Money Transfer locations
for idx, row in WUMoneyTransferLoc_df.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=WUMoneyTransferLoc_df['Name'].loc[WUMoneyTransferLoc_df.index[idx]], icon=folium.Icon(color='red')).add_to(m_10)

# Show the map
m_10

Mex Express (Intermex?) Locations

In [ ]:
MELoc_df = pd.read_csv("MexExpressLoc.csv")
MELoc_df.head()

,Name,Address
0,Mex Express,"3070 24th St, San Francisco, CA 94110"
1,Mex Express,"2141 Mission St # B, San Francisco, CA 94110"
2,Mex Express,"446 Eddy St, San Francisco, CA 94109"
3,Mex Express,"2390 Mission St #105, San Francisco, CA 94110"


In [ ]:
MELat = []
MELon = []
for index, row in MELoc_df.iterrows():
  location = geolocator.geocode(WULoc_df['Address'].loc[WULoc_df.index[index]])
  MELat.append(location.latitude)
  MELon.append(location.longitude)

MELoc_df["Latitude"] = MELat
MELoc_df["Longitude"] = MELon

MELoc_df.head()

,Name,Address,Latitude,Longitude
0,Mex Express,"3070 24th St, San Francisco, CA 94110",37.765371,-122.443814
1,Mex Express,"2141 Mission St # B, San Francisco, CA 94110",37.764405,-122.452336
2,Mex Express,"446 Eddy St, San Francisco, CA 94109",37.761010,-122.435204
3,Mex Express,"2390 Mission St #105, San Francisco, CA 94110",37.761829,-122.434842


In [ ]:
# Create map Zoomed in on Mission District, SF
m_7 = folium.Map(location=[37.76, -122.42], zoom_start=14)

# Visualize Mission District
for _, r in mission_df.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    geo_j.add_to(m_7)

# Visualize the Western Union locations
for idx, row in MELoc_df.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=MELoc_df['Name'].loc[MELoc_df.index[idx]], icon=folium.Icon(color='orange')).add_to(m_7)

# Show the map
m_7

Saving Map as HTML file (Open w/ Chrome)

In [ ]:
# Save files
m_2.save("San Francisco NGOs.html")
m_10.save("San Francisco Western Unions.html")
m_7.save("San Francisco Mex Express.html")

In [ ]:
#highest_demand = outside_range.geometry.apply(best_X).value_counts().idxmax()